<a href="https://colab.research.google.com/github/narendra1711/loan_prediction/blob/master/loan_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installed Below Libraries

In [1]:
'''
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sklearn
'''

'\n!pip install pandas\n!pip install numpy\n!pip install matplotlib\n!pip install seaborn\n!pip install sklearn\n'

## Import Below Libraries

In [0]:
#gereral libraries
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#libraries for logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#libraries for model evaluation
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    f1_score,
    auc,
    average_precision_score)

#libraries for KNN Classifier
from sklearn.neighbors import KNeighborsClassifier

#ignore warnings
import warnings
warnings.filterwarnings('ignore')